In [1]:
import pandas as pd
import numpy as np

In [2]:
recommended_list = [143, 156, 1134, 991, 27, 1543, 3345, 533, 11, 43] #id товаров
bought_list = [521, 32, 143, 991]

### 1. Hit rate

In [4]:
def hit_rate(recommended_list, bought_list):
    
    bought_list = np.array(bought_list)
    recommended_list = np.array(recommended_list)
    
    flags = np.isin(bought_list, recommended_list)
    
    hit_rate = (flags.sum() > 0) * 1
    
    return hit_rate

In [5]:
hit_rate(recommended_list, bought_list)

1

In [17]:
def hit_rate_at_k(recommended_list, bought_list, k):
    
    bought_list = np.array(bought_list)
    recommended_list = np.array(recommended_list)
    
    flags = np.isin(bought_list, recommended_list[:k])
    
    hit_rate = (flags.sum() > 0) * 1
    
    return hit_rate

In [22]:
hit_rate_at_k(recommended_list, bought_list, 8)

1

### 2. Precision

In [23]:
def precision(recommended_list, bought_list):
    
    bought_list = np.array(bought_list)
    recommended_list = np.array(recommended_list)
    
    flags = np.isin(bought_list, recommended_list)
    
    precision = flags.sum() / len(recommended_list)
    
    return precision

In [39]:
def precision_at_k(recommended_list, bought_list, k):
    
    bought_list = np.array(bought_list)
    recommended_list = np.array(recommended_list)
    
    #assert len(bought_list) > len(recommended_list)
    bought_list = bought_list  # Тут нет [:k] !!
    recommended_list = recommended_list[:k]
    
    flags = np.isin(bought_list, recommended_list)
    
    precision = flags.sum() / len(recommended_list)
    
    
    return precision

In [61]:
def money_precision_at_k(recommended_list, bought_list, prices_recommended, k):
        
    bought_list = np.array(bought_list)
    recommended_list = np.array(recommended_list[:k])
    prices_recommended = np.array(prices_recommended[:k])
    
    flags = np.isin(recommended_list, bought_list)
    
    precision = np.dot(prices_recommended, flags) / prices_recommended.sum()
    
    return precision

In [62]:
prices_recommended = [1, 1, 10, 1, 1, 1, 1, 1, 1]

In [63]:
money_precision_at_k(recommended_list, bought_list, prices_recommended, k=4)

0.8461538461538461

In [ ]:
def average_precision (recommended_list, bought_list):
    bought_list = np.array(bought_list)
    recommended_list = np.array(recommended_list)
    
    summ = 0
    relevant = 0
    for i in range(len(recommended_list)):
        if recommended_list[i] in bought_list:
            relevant += 1
            summ = summ + relevant / (i + 1)
            
    return summ/relevant

In [ ]:
average_precision (recommended_list, bought_list)

### 3. Recall

In [71]:
def recall(recommended_list, bought_list):
    
    bought_list = np.array(bought_list)
    recommended_list = np.array(recommended_list)
    
    flags = np.isin(bought_list, recommended_list)
    
    recall = flags.sum() / len(bought_list)
    
    return recall


def recall_at_k(recommended_list, bought_list, k=5):
    
    bought_list = np.array(bought_list)
    recommended_list = np.array(recommended_list[:k])
    
    flags = np.isin(bought_list, recommended_list)
    
    recall = flags.sum() / len(bought_list)
    
    return recall


def money_recall_at_k(recommended_list, bought_list, prices_recommended, prices_bought, k=5):
    
    bought_list = np.array(bought_list)
    recommended_list = np.array(recommended_list[:k])
    prices_recommended = np.array(prices_recommended[:k])
    prices_bought = np.array(prices_bought)
    
    flags = np.isin(recommended_list, bought_list)
    
    recall = np.dot(prices_recommended, flags) / prices_bought.sum()
    
    return recall

In [70]:
recall_at_k(recommended_list, bought_list, 4)

0.25

In [75]:
prices_bought  = [5, 1, 10, 10] #цена купленных товаров
recommended_list = [1, 2, 3, 4, 5, 6, 7, 8, 9] #id товаров
bought_list = [1, 5, 12, 15] #купленные товары
prices_recommended = [5, 1, 10, 1, 1, 1, 1, 1, 1] #цена рекомендованных товаров

In [76]:
money_recall_at_k(recommended_list, bought_list, prices_recommended, prices_bought, k=5)

0.23076923076923078

In [ ]:
recommended_list = [1, 2, 3, 4, 5, 6, 7, 8, 9]
bought_list = [1, 5, 12, 15]

### MAP@k

MAP@k (Mean Average Precision@k)  
Среднее AP@k по всем юзерам
- Показывает средневзвешенную точность рекомендаций

In [ ]:
def ap_k(recommended_list, bought_list, k=5):
    
    bought_list = np.array(bought_list)
    recommended_list = np.array(recommended_list)
    
    flags = np.isin(recommended_list, bought_list)
    
    if sum(flags) == 0:
        return 0
    
    sum_ = 0
    for i in range(0, k-1):
        if flags[i] == True:
            p_k = precision_at_k(recommended_list, bought_list, k=i+1)
            sum_ += p_k
            
    result = sum_ / sum(flags)
    
    return result

def ap_k_(recommended_list, bought_list):
    
    flags = np.isin(recommended_list, bought_list)
    
    if sum(flags) == 0:
        return 0
    
    sum_ = 0
    for i in range(0, k-1):
        if flags[i] == True:
            p_k = precision_at_k(recommended_list, bought_list, k=i+1)
            sum_ += p_k
            
    result = sum_ / sum(flags)
    
    return result

def map_k(count_user = 1, *arg): #ожидается, что будут введены поочередно recommended_list, затем bought_list по кажому пользователю
    
    summ = 0
    
    for i in range(count_user):
        bought_list = np.array(arg)
        recommended_list = np.array(arg)
        
        summ += ap_k_(recommended_list, bought_list)
        
    result = summ / count_user
    
    return result
#Не оттестированно. Жалко что нет примеров в лекции, на которых можно было бы точно проверить работу алгоритма. 
#Понимание вроде есть, но смутное
#И *arg впервые использую в самостоятельной разработке, раньше более по наитию и примерам. Но по моему так надо делать.
        